In [ ]:
'''pod communication '''
'''node delay metrix '''

''''check under resource limits'''

In [1]:
from kubernetes import client, config, stream
import pandas as pd
import matplotlib.pyplot as plt

from prometheus_api_client import PrometheusConnect

from datetime import datetime, timedelta
from difflib import diff_bytes

import seaborn as sns
import numpy as np
import random
import multiprocessing as mp
from timeit import default_timer as timer
from datetime import datetime, timedelta

# Kubernetes Config
config.load_kube_config()
v1 = client.CoreV1Api()

# Prometheus Config
#prom_url = "http://<PROMETHEUS_SERVER_IP>:<PORT>"
# prom_url = "http://10.110.188.57:9090"
prom_url = "http://10.105.116.175:9090"

prom = PrometheusConnect(url=prom_url, disable_ssl=True)
#test prom connection
prom_connect_response = prom.custom_query(query="up")
print(prom_connect_response)

[{'metric': {'__name__': 'up', 'app': 'details', 'instance': '192.168.231.244:15020', 'job': 'kubernetes-pods', 'namespace': 'default', 'pod': 'details-v1-6cccc87c79-gwpkf', 'pod_template_hash': '6cccc87c79', 'security_istio_io_tlsMode': 'istio', 'service_istio_io_canonical_name': 'details', 'service_istio_io_canonical_revision': 'v1', 'version': 'v1'}, 'value': [1728023133.252, '1']}, {'metric': {'__name__': 'up', 'app': 'istio-egressgateway', 'chart': 'gateways', 'heritage': 'Tiller', 'install_operator_istio_io_owning_resource': 'unknown', 'instance': '192.168.231.238:15020', 'istio': 'egressgateway', 'istio_io_rev': 'default', 'job': 'kubernetes-pods', 'namespace': 'istio-system', 'operator_istio_io_component': 'EgressGateways', 'pod': 'istio-egressgateway-687cb674fc-sbsw5', 'pod_template_hash': '687cb674fc', 'release': 'istio', 'service_istio_io_canonical_name': 'istio-egressgateway', 'service_istio_io_canonical_revision': 'latest', 'sidecar_istio_io_inject': 'false'}, 'value': [17

In [2]:
'''trigger decision'''

import time
from datetime import datetime, timedelta
from urllib import response

# Configurable parameters
QoS_target = 300  # QoS target (ms) from knee point analysis, specific to the application
time_window = 1  # Time window in minutes, for 'rate' istio function
_Namespace = 'social-network3'
response_code = '200'  # Consider only successful responses (HTTP 200), to get successful response time

# Define the time range for the query
# check_start_time = datetime(2024, 7, 11, 4, 17, 0) # UTC time
# check_end_time = datetime(2024, 7, 11, 4, 35, 0)
# end_time = datetime.now() - timedelta(minutes=2)  # Current time minus ten minutes,,  - timedelta(minutes=10)
# start_time = end_time - timedelta(minutes=time_window)  # Start time is 'time_window' minutes before end time
start_time = datetime(2024, 7, 14, 12, 57, 0) # UTC time
end_time = datetime(2024, 7, 14, 13, 16, 0)

# start_time = datetime(2024, 7, 11, 4, 10, 0) # UTC time
# end_time = datetime(2024, 7, 11, 4, 59, 0)

def trigger_migration(QoS_target, time_window, app_namespace):
    trigger = False
    step = str(time_window * 60)  # Step size in seconds for Prometheus queries

    # Prometheus queries for Istio metrics
    istio_request_duration_query = f'rate(istio_request_duration_milliseconds_sum{{namespace="{app_namespace}", app="nginx-thrift", response_code="{response_code}"}}[{1}m])'
    istio_requests_total_query = f'rate(istio_requests_total{{namespace="{app_namespace}", app="nginx-thrift", response_code="{response_code}"}}[{1}m])'

    # Fetch the data from Prometheus
    istio_request_duration_response = prom.custom_query_range(
        query=istio_request_duration_query,
        start_time=start_time,
        end_time=end_time,
        step=step
    )
    istio_requests_total_response = prom.custom_query_range(
        query=istio_requests_total_query,
        start_time=start_time,
        end_time=end_time,
        step=step
    )

    # Ensure there is data to process
    if istio_request_duration_response and istio_requests_total_response:
        duration_values = [float(val[1]) for val in istio_request_duration_response[0]['values']]
        total_requests_values = [float(val[1]) for val in istio_requests_total_response[0]['values']]
        print(duration_values)
        print(total_requests_values)

        if total_requests_values and duration_values:
            # Compute the average response time in milliseconds
            total_requests_sum = sum(total_requests_values)
            if total_requests_sum > 0:
                average_response_time = sum(duration_values) / total_requests_sum
                print(f"Average response time = {average_response_time:.2f} ms")

                # Check if the average response time exceeds the QoS target
                if average_response_time > QoS_target:
                    trigger = True
            else:
                print("Total requests sum is zero, cannot compute average response time.")
        else:
            print("No traffic detected, no trigger.")
    else:
        print("Query returned no data, no trigger.")

    
    print(f"Trigger = {trigger}")

    return trigger

# Example usage
trigger_migration(QoS_target=QoS_target, time_window=time_window, app_namespace=_Namespace)


[0.0, 0.0, 0.0, 0.0, 10277.144444444442, 89994.09999999999, 0.0, 0.0, 0.0, 0.0, 100672.88888888888, 100060.97777777776, 0.0, 0.0, 0.0, 0.0, 100765.09999999999, 98786.66666666666, 0.0, 0.0]
[0.0, 0.0, 0.0, 10.799999999999999, 193.1333333333333, 184.77777777777774, 0.0, 0.0, 0.0, 14.355555555555554, 190.95555555555552, 170.84444444444443, 0.0, 0.0, 0.0, 19.688888888888886, 183.59999999999997, 171.95555555555555, 0.0, 0.0]
Average response time = 439.04 ms
Trigger = True


True

In [22]:
'''set global resource limits to every worker node:
sum_CPU_Limits < 80% of the each node's cpu;
sum_requested_CPU < 40% of the each node's memory; 

sum_memory_Limits < 70%
sum_requested_memory < 70%

'''

'set global resource limits to every worker node:'

In [3]:
'''Building traffic Stress graph, return the ms execution graph with bi-direction traffic '''
# Function to retrieve ready deployments
def get_ready_deployments(namespace):
    ready_deployments = []
    deployments = client.AppsV1Api().list_namespaced_deployment(namespace)
    for deployment in deployments.items:
        if deployment.status.ready_replicas == deployment.spec.replicas:
            ready_deployments.append(deployment.metadata.name)
    return ready_deployments

# Function to calculate transmitted requests
def transmitted_req_calculator(workload_src, workload_dst, timerange, step_interval, app_namespace):
    # end_time = datetime.now()
    # start_time = end_time - timedelta(minutes=timerange)
    start_time = datetime(2024, 7, 11, 4, 10, 0) # UTC time
    end_time = datetime(2024, 7, 11, 4, 59, 0)

    istio_tcp_sent_query = f'istio_tcp_sent_bytes_total{{reporter="source",source_workload="{workload_src}",destination_workload="{workload_dst}", namespace = "{app_namespace}"}}'
    istio_tcp_received_query = f'istio_tcp_received_bytes_total{{reporter="source",source_workload="{workload_src}",destination_workload="{workload_dst}", namespace = "{app_namespace}"}}'

    istio_tcp_sent_response = prom.custom_query_range(
        query=istio_tcp_sent_query,
        start_time=start_time,
        end_time=end_time,
        step=step_interval
    )
    istio_tcp_received_response = prom.custom_query_range(
        query=istio_tcp_received_query,
        start_time=start_time,
        end_time=end_time,
        step=step_interval
    )

    # only calculate the from {workload_src} to {workload_dst} that have no traffic.
    if (not istio_tcp_sent_response or not istio_tcp_sent_response[0]['values']) and (not istio_tcp_received_response or not istio_tcp_received_response[0]['values']):
        # print(f'from {workload_src} to {workload_dst} average_traffic_bytes: no values' )
        return 0
    else:
        values_sent = istio_tcp_sent_response[0]['values']
        values_received = istio_tcp_received_response[0]['values']

        begin_timestamp, begin_traffic_sent_counter = values_sent[0]
        end_timestamp, end_traffic_sent_counter = values_sent[-1]
        begin_timestamp, begin_traffic_received_counter = values_received[0]
        end_timestamp, end_traffic_received_counter = values_received[-1]

        data_points_num_sent = len(values_sent)
        data_points_num_recevied = len(values_received)

        average_traffic_sent = (int(end_traffic_sent_counter) - int(begin_traffic_sent_counter)) / data_points_num_sent
        average_traffic_received = (int(end_traffic_received_counter) - int(begin_traffic_received_counter)) / data_points_num_recevied

        average_traffic_bytes = int((average_traffic_sent + average_traffic_received) / 2)
        print(f'from {workload_src} to {workload_dst} average_traffic_bytes: {int(average_traffic_bytes/1000)} KB' )
        return average_traffic_bytes
# Retrieve the list of ready deployments
_Namespace = 'social-network3'
ready_deployments = get_ready_deployments(_Namespace)

# Initialize an empty DataFrame for the exec_graph
df_exec_graph = pd.DataFrame(index=ready_deployments, columns=ready_deployments, data=0.0)


# Fill the DataFrame with average request values
for deployment_src in ready_deployments:
    for deployment_dst in ready_deployments:
        if deployment_src != deployment_dst:
            average_requests = transmitted_req_calculator(
                workload_src=deployment_src, 
                workload_dst=deployment_dst, 
                timerange=10,  # look back at the 10mins long history
                step_interval='1m', # window step in 1 miniute
                app_namespace = _Namespace
            )
            df_exec_graph.at[deployment_src, deployment_dst] = int(average_requests/1000) # calcuate KiloByte, instead of Byte

# Plot the heatmap for exec_graph
# plt.figure(figsize=(15, 12))
# sns.heatmap(df_exec_graph, cmap='viridis', fmt=".2f")
# plt.title('Average Requests per 5 Minutes among Pods')
# plt.xlabel('Destination Pods')
# plt.ylabel('Source Pods')
# plt.show()

# Convert DataFrame to exec_graph (numpy array)
exec_graph = df_exec_graph.to_numpy()


from compose-post-service to home-timeline-service average_traffic_bytes: 21 KB
from compose-post-service to media-service average_traffic_bytes: 30 KB
from compose-post-service to post-storage-service average_traffic_bytes: 118 KB
from compose-post-service to text-service average_traffic_bytes: 162 KB
from compose-post-service to unique-id-service average_traffic_bytes: 18 KB
from compose-post-service to user-service average_traffic_bytes: 25 KB
from compose-post-service to user-timeline-service average_traffic_bytes: 20 KB
from home-timeline-service to home-timeline-redis average_traffic_bytes: 24 KB
from home-timeline-service to post-storage-service average_traffic_bytes: 81 KB
from home-timeline-service to social-graph-service average_traffic_bytes: 17 KB
from nginx-thrift to compose-post-service average_traffic_bytes: 87 KB
from nginx-thrift to home-timeline-service average_traffic_bytes: 97 KB
from nginx-thrift to user-timeline-service average_traffic_bytes: 90 KB
from post-stora

In [4]:
df_exec_graph.to_csv('df_exec_graph.csv')

In [5]:
deployment_node_dict = {}
deployment_list = ready_deployments
apps_v1 = client.AppsV1Api()


for deployment_name in deployment_list:
    try:
        # Get the deployment object
        deployment = apps_v1.read_namespaced_deployment(deployment_name, namespace='social-network3')
        # Get the pod selector from the deployment
        pod_selector = deployment.spec.selector.match_labels

        # List all pods in the namespace
        pods = v1.list_namespaced_pod(_Namespace)

        # Find pods that match the deployment selector
        for pod in pods.items:
            if all(item in pod.metadata.labels.items() for item in pod_selector.items()):
                # Get the node name where the pod is running
                node_name = pod.spec.node_name
                # Assuming one pod per deployment for simplicity, can be expanded if needed
                deployment_node_dict[deployment_name] = node_name
                break  # Found the pod for this deployment, no need to continue

    except client.exceptions.ApiException as e:
        print(f"Exception when retrieving deployment {deployment_name}: {e}")




# return 
deployment_node_dict


{'compose-post-service': 'k8s-worker-1',
 'home-timeline-redis': 'k8s-worker-1',
 'home-timeline-service': 'k8s-worker-1',
 'jaeger': 'k8s-worker-2',
 'media-frontend': 'k8s-worker-6',
 'media-memcached': 'k8s-worker-1',
 'media-mongodb': 'k8s-worker-4',
 'media-service': 'k8s-worker-1',
 'nginx-thrift': 'k8s-worker-1',
 'post-storage-memcached': 'k8s-worker-5',
 'post-storage-mongodb': 'k8s-worker-5',
 'post-storage-service': 'k8s-worker-4',
 'social-graph-mongodb': 'k8s-worker-1',
 'social-graph-redis': 'k8s-worker-1',
 'social-graph-service': 'k8s-worker-1',
 'text-service': 'k8s-worker-1',
 'unique-id-service': 'k8s-worker-1',
 'url-shorten-memcached': 'k8s-worker-6',
 'url-shorten-mongodb': 'k8s-worker-6',
 'url-shorten-service': 'k8s-worker-9',
 'user-memcached': 'k8s-worker-1',
 'user-mention-service': 'k8s-worker-1',
 'user-mongodb': 'k8s-worker-1',
 'user-service': 'k8s-worker-1',
 'user-timeline-mongodb': 'k8s-worker-2',
 'user-timeline-redis': 'k8s-worker-4',
 'user-timeline

In [6]:
def get_worker_node_numbers(deployment_node_dict):
    # Extract node numbers from the dictionary
    
    #the actual node number is like (node1, node2, node3, ...., node9)
    # node_numbers = [int(node.split('-')[-1]) for node in deployment_node_dict.values()] 
    
    # make the correspoding adjustment to (node0, node1, node2, ...,node8), otherwsie there will be index>=9 error in total_cost calculation
    node_numbers = [(int(node.split('-')[-1]) - 1) for node in deployment_node_dict.values()] 

    return node_numbers

get_worker_node_numbers(deployment_node_dict) # eg. k8s-worker-1 will be numbered as 0, k8s-worker-2 will be numbered as 1, etc.

[0,
 0,
 0,
 1,
 5,
 0,
 3,
 0,
 0,
 4,
 4,
 3,
 0,
 0,
 0,
 0,
 0,
 5,
 5,
 8,
 0,
 0,
 0,
 0,
 1,
 3,
 1]

In [7]:
df_exec_graph

,compose-post-service,home-timeline-redis,home-timeline-service,jaeger,media-frontend,media-memcached,media-mongodb,media-service,nginx-thrift,post-storage-memcached,...,url-shorten-memcached,url-shorten-mongodb,url-shorten-service,user-memcached,user-mention-service,user-mongodb,user-service,user-timeline-mongodb,user-timeline-redis,user-timeline-service
compose-post-service,0.0,0.0,21.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,20.0
home-timeline-redis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
home-timeline-service,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
jaeger,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
media-frontend,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
media-memcached,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
media-mongodb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
media-service,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nginx-thrift,87.0,0.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0
post-storage-memcached,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
'''get the cross-node delay matrix'''


from kubernetes import client, config, stream
import concurrent.futures

# Handle individual latency measurements between source and target pods.
def measure_latency_from_source_to_target(v1, namespace, source_pod, target_pod):
    source_pod_name = source_pod.metadata.name
    source_pod_node_name = source_pod.spec.node_name
    target_pod_ip = target_pod.status.pod_ip
    target_pod_name = target_pod.metadata.name
    target_pod_node_name = target_pod.spec.node_name
    result = (source_pod_node_name, target_pod_node_name, None)

    if source_pod_name != target_pod_name:
        exec_command = ['curl', '-o', '/dev/null', '-s', '-w', '%{time_total}', f'http://{target_pod_ip}']
        latencies = []
        for _ in range(5):
            try:
                resp = stream.stream(v1.connect_get_namespaced_pod_exec,
                                     source_pod_name,
                                     namespace,
                                     command=exec_command,
                                     stderr=True,
                                     stdin=False,
                                     stdout=True,
                                     tty=False)
                latency = float(resp) * 1000  # Convert the measured seconds into milliseconds
                latencies.append(latency)
            except Exception as e:
                print(f"Error executing command in pod {source_pod_name}: {e}")
        
        if latencies:
            min_latency = min(latencies) #choose the lowest measured vaule to represent the latency
            result = (source_pod_node_name, target_pod_node_name, min_latency)
            print(f"Latency from source node {source_pod_node_name} to target node {target_pod_node_name}: {min_latency} milliseconds")
        else:
            result = (source_pod_node_name, target_pod_node_name, "Error")

    return result

def measure_http_latency(namespace='measure-nodes'):
    v1 = client.CoreV1Api()
    pods = v1.list_namespaced_pod(namespace, label_selector="app=latency-measurement").items
    latency_results = {}
    
    # Run the latency measurement tasks concurrently
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for source_pod in pods:
            for target_pod in pods:
                futures.append(executor.submit(measure_latency_from_source_to_target, v1, namespace, source_pod, target_pod))
        
        # Aggregate the completed results into the latency_results dictionary
        for future in concurrent.futures.as_completed(futures):
            source_pod_node_name, target_pod_node_name, latency = future.result()
            if source_pod_node_name not in latency_results:
                latency_results[source_pod_node_name] = {}
            latency_results[source_pod_node_name][target_pod_node_name] = latency

    return latency_results

# Call the function to measure latency after deploying the DaemonSet
namespace = 'measure-nodes'
# After the DaemonSet is ready, and the related pods are ready
latency_results = measure_http_latency(namespace=namespace)
# print(latency_results)



# Convert the nested dictionary into a pandas DataFrame and handle self-latency
df_latency = pd.DataFrame(latency_results).T
for worker in df_latency.columns:
    df_latency.at[worker, worker] = 0
    
# Convert DataFrame to delay matrix (numpy array)
delay_matrix = df_latency.to_numpy()
delay_matrix


Latency from source node k8s-worker-6 to target node k8s-worker-5: 15.691 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-5: 29.619 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-8: 17.055 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-1: 13.013 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-6: 29.678 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-7: 14.784 milliseconds
Latency from source node k8s-worker-6 to target node k8s-worker-3: 29.596 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-2: 16.787 milliseconds
Latency from source node k8s-worker-6 to target node k8s-worker-9: 13.733 milliseconds
Latency from source node k8s-worker-3 to target node k8s-worker-9: 55.963 milliseconds
Latency from source node k8s-worker-9 to target node k8s-worker-7: 16.87 milliseconds
Latency from source node k8s-worker-7 to tar

array([[ 0.   , 29.619, 17.055, 13.013, 29.678, 14.784, 16.787, 55.963,
        15.936],
       [14.769, 18.708, 15.848, 47.59 , 10.784,  0.   , 48.511, 16.945,
        31.606],
       [29.596, 15.691, 25.678, 23.954,  0.   , 10.895, 45.056, 13.733,
        22.793],
       [56.224, 34.87 , 13.028, 58.139, 14.002, 16.87 , 52.32 ,  0.   ,
        35.097],
       [29.592,  0.   , 16.724, 34.825, 15.622, 18.997, 26.676, 34.866,
        16.258],
       [12.927, 34.932, 34.793,  0.   , 23.899, 47.536, 11.626, 58.134,
        27.694],
       [16.628, 26.683, 56.688, 11.467, 44.649, 48.573,  0.   , 52.002,
        28.531],
       [16.72 , 16.656,  0.   , 34.944, 26.099, 15.855, 56.817, 13.06 ,
        23.881],
       [15.834, 15.967, 23.678, 27.964, 23.284, 31.756, 28.562, 35.028,
         0.   ]])

In [9]:
delay_matrix

array([[ 0.   , 29.619, 17.055, 13.013, 29.678, 14.784, 16.787, 55.963,
        15.936],
       [14.769, 18.708, 15.848, 47.59 , 10.784,  0.   , 48.511, 16.945,
        31.606],
       [29.596, 15.691, 25.678, 23.954,  0.   , 10.895, 45.056, 13.733,
        22.793],
       [56.224, 34.87 , 13.028, 58.139, 14.002, 16.87 , 52.32 ,  0.   ,
        35.097],
       [29.592,  0.   , 16.724, 34.825, 15.622, 18.997, 26.676, 34.866,
        16.258],
       [12.927, 34.932, 34.793,  0.   , 23.899, 47.536, 11.626, 58.134,
        27.694],
       [16.628, 26.683, 56.688, 11.467, 44.649, 48.573,  0.   , 52.002,
        28.531],
       [16.72 , 16.656,  0.   , 34.944, 26.099, 15.855, 56.817, 13.06 ,
        23.881],
       [15.834, 15.967, 23.678, 27.964, 23.284, 31.756, 28.562, 35.028,
         0.   ]])

In [30]:
df_latency.to_csv('df_latency.csv')

In [10]:
df_latency

,k8s-worker-3,k8s-worker-5,k8s-worker-8,k8s-worker-1,k8s-worker-6,k8s-worker-7,k8s-worker-2,k8s-worker-9,k8s-worker-4
k8s-worker-3,0.000,29.619,17.055,13.013,29.678,14.784,16.787,55.963,15.936
k8s-worker-7,14.769,18.708,15.848,47.590,10.784,0.000,48.511,16.945,31.606
k8s-worker-6,29.596,15.691,25.678,23.954,0.000,10.895,45.056,13.733,22.793
k8s-worker-9,56.224,34.870,13.028,58.139,14.002,16.870,52.320,0.000,35.097
k8s-worker-5,29.592,0.000,16.724,34.825,15.622,18.997,26.676,34.866,16.258
k8s-worker-1,12.927,34.932,34.793,0.000,23.899,47.536,11.626,58.134,27.694
k8s-worker-2,16.628,26.683,56.688,11.467,44.649,48.573,0.000,52.002,28.531
k8s-worker-8,16.720,16.656,0.000,34.944,26.099,15.855,56.817,13.060,23.881
k8s-worker-4,15.834,15.967,23.678,27.964,23.284,31.756,28.562,35.028,0.000


In [11]:
'''microservice-node mapping placement'''
# import seaborn as sns
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import random
import multiprocessing as mp
# from timeit import default_timer as timer
# Define the functions for placement and cost calculation
def calculate_communication_cost(exec_graph, placement, delay_matrix):
    cost = 0
    for u in range(len(exec_graph)):
        for v in range(len(exec_graph[u])):
            if exec_graph[u][v] > 0:
                server_u = placement[u]
                server_v = placement[v]
                cost += exec_graph[u][v] * delay_matrix[server_u][server_v]
    return cost

def greedy_placement_worker(exec_graph, delay_matrix, placement, num_servers, start, end):
    current_cost = calculate_communication_cost(exec_graph, placement, delay_matrix)
    improved = True
    while improved:
        improved = False
        for u in range(start, end):
            current_server = placement[u]
            for new_server in range(num_servers):
                if new_server != current_server:
                    new_placement = placement.copy()
                    new_placement[u] = new_server
                    new_cost = calculate_communication_cost(exec_graph, new_placement, delay_matrix)
                    if new_cost < current_cost:
                        placement = new_placement
                        current_cost = new_cost
                        improved = True
                        break
            if improved:
                break
    return placement, current_cost

# divides the placement optimization task into chunks and runs them in parallel using multiple worker processes.
def parallel_greedy_placement(exec_graph, delay_matrix, placement, num_servers, num_workers=4):
    num_microservices = len(exec_graph)
    chunk_size = (num_microservices + num_workers - 1) // num_workers  # Adjust chunk size to cover all microservices
    print(f"{chunk_size} = ({num_microservices} + {num_workers} - 1) // {num_workers}")

    while True:
        pool = mp.Pool(num_workers)
        chunks = [(exec_graph, delay_matrix, placement, num_servers, i*chunk_size, min((i+1)*chunk_size, num_microservices)) for i in range(num_workers)] #divides the placement optimization task into chunks
        results = pool.starmap(greedy_placement_worker, chunks) #distributes the chunks to the worker processes and collects their results
        pool.close() #prevents any more tasks from being submitted to the pool
        pool.join() #waits for all worker processes to finish

        # print("Chunks:", chunks)
        print("lengnths of results:", len(results))
        print("Results:", results)
        # Find the best result from all chunks
        new_placement = results[0][0] #initializes the best placement and cost with the first result
        new_cost = results[0][1] #initializes the best placement and cost with the first result
        improved = False

        for result in results[1:]:
            if result[1] < new_cost:
                new_placement = result[0]
                new_cost = result[1]
                improved = True

        if not improved:
            break

        placement = new_placement

    return placement, new_cost


# test usage with the generated delay matrix and initial placement
M = len(exec_graph)  # Number of microservices
N = len(delay_matrix)  # Number of servers

# Initial random placement
# placement = [random.randint(0, N - 1) for _ in range(M)]
initial_placement = get_worker_node_numbers(deployment_node_dict)
print("innitial_placement:", initial_placement)


initial_cost = calculate_communication_cost(exec_graph, initial_placement, delay_matrix)

# Perform parallel greedy placement
final_placement, total_cost = parallel_greedy_placement(exec_graph, delay_matrix, initial_placement, N, num_workers= mp.cpu_count()) #num_workers=mp.cpu_count()


print("initial_cost:", initial_cost)
print("Final Placement:", final_placement)
print("Total Communication Cost:", total_cost)


innitial_placement: [0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1]
1 = (27 + 32 - 1) // 32
lengnths of results: 32
Results: [([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1], 20624.825), ([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1], 20624.825), ([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1], 20624.825), ([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1], 20624.825), ([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1], 20624.825), ([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1], 20624.825), ([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1], 20624.825), ([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0, 1, 3, 1], 20624.825), ([0, 0, 0, 1, 5, 0, 3, 0, 0, 4, 4, 3, 0, 0, 0, 0, 0, 5, 5, 8, 0, 0, 0, 0

In [12]:
ready_deployments

['compose-post-service',
 'home-timeline-redis',
 'home-timeline-service',
 'jaeger',
 'media-frontend',
 'media-memcached',
 'media-mongodb',
 'media-service',
 'nginx-thrift',
 'post-storage-memcached',
 'post-storage-mongodb',
 'post-storage-service',
 'social-graph-mongodb',
 'social-graph-redis',
 'social-graph-service',
 'text-service',
 'unique-id-service',
 'url-shorten-memcached',
 'url-shorten-mongodb',
 'url-shorten-service',
 'user-memcached',
 'user-mention-service',
 'user-mongodb',
 'user-service',
 'user-timeline-mongodb',
 'user-timeline-redis',
 'user-timeline-service']

In [13]:
''' QoS traget not meet, and if triggered migration
(1) measure the ms graph with traffic links, get the graph (stress graph)
(2) measure the cross-node latency metrix
(3) run the service-node mapping algorithm

(4) load-aware monitor 
    if sepcific node is overloaded, migrate the pods to other less overloaded node
        maintain a overhead table.

'''
import pod_migration as pm

# compare the initial and final placement, find the migrations needed
def migrate_microservices(initial_placement, final_placement):
    migrations = []
    for microservice, (initial, final) in enumerate(zip(initial_placement, final_placement)):
        if initial != final:
            migrations.append((microservice, initial, final))
    return migrations

def exclude_non_App_ms(migrations, microservice_names, exclude_deployments=['jager']):
    excluded_indices = {index for index, name in enumerate(microservice_names) if name in exclude_deployments}
    return [(ms, initial, final) for ms, initial, final in migrations if ms not in excluded_indices]



# Avoid migrating non-application deployments like 'jager' and 'prometheus'
exclude_deployments = ['jaeger']
microservice_names = ready_deployments

migrations = migrate_microservices(initial_placement, final_placement)
filtered_migrations = exclude_non_App_ms(migrations, microservice_names, exclude_deployments)





print("Migrations needed:")
for microservice, initial, final in filtered_migrations: # iterate over the filtered migrations
    print(f"----Microservice_Num {microservice} from Node {initial} to Node {final}----")
    print(f"Microservice {ready_deployments[microservice]} from Node {initial+1} to Node {final+1}")
    if pm.patch_deployment(ready_deployments[microservice], _Namespace, new_node_name='k8s-worker-' + str(final + 1)):
        pm.wait_for_rolling_update_to_complete(ready_deployments[microservice], _Namespace, new_node_name='k8s-worker-' + str(final + 1))
        print(f"Microservice {ready_deployments[microservice]} migrated successfully.")
    
    
    




# import time
# import pod_migration as pm

# while(True):
#     if(trigger_migration):
#         # just migrate application pods, exclude the jager and prometheus pods
#         stress_graph =[[]]
        
        
        
 
#     time.sleep(60*5) # check every 5 mins





Migrations needed:
----Microservice_Num 9 from Node 4 to Node 7----
Microservice post-storage-memcached from Node 5 to Node 8
Deployment 'post-storage-memcached' patched to schedule pods on 'k8s-worker-8'.
Waiting for the rolling update to complete...
all_pods_updated= False
len(pods)= 1
Rolling update in progress...
all_pods_updated= False
len(pods)= 2
Rolling update in progress...
all_pods_updated= False
len(pods)= 2
Rolling update in progress...
all_pods_updated= False
len(pods)= 2
Rolling update in progress...
all_pods_updated= True
len(pods)= 1
All pods are running on the new node.
Microservice post-storage-memcached migrated successfully.
----Microservice_Num 10 from Node 4 to Node 7----
Microservice post-storage-mongodb from Node 5 to Node 8
Deployment 'post-storage-mongodb' patched to schedule pods on 'k8s-worker-8'.
Waiting for the rolling update to complete...
all_pods_updated= False
len(pods)= 1
Rolling update in progress...
all_pods_updated= False
len(pods)= 2
Rolling updat

In [21]:
deployment_node_dict

{'compose-post-service': 'k8s-worker-1',
 'home-timeline-redis': 'k8s-worker-4',
 'home-timeline-service': 'k8s-worker-7',
 'jaeger': 'k8s-worker-2',
 'media-frontend': 'k8s-worker-6',
 'media-memcached': 'k8s-worker-1',
 'media-mongodb': 'k8s-worker-4',
 'media-service': 'k8s-worker-2',
 'nginx-thrift': 'k8s-worker-4',
 'post-storage-memcached': 'k8s-worker-7',
 'post-storage-mongodb': 'k8s-worker-8',
 'post-storage-service': 'k8s-worker-4',
 'social-graph-mongodb': 'k8s-worker-1',
 'social-graph-redis': 'k8s-worker-9',
 'social-graph-service': 'k8s-worker-1',
 'text-service': 'k8s-worker-2',
 'unique-id-service': 'k8s-worker-2',
 'url-shorten-memcached': 'k8s-worker-6',
 'url-shorten-mongodb': 'k8s-worker-6',
 'url-shorten-service': 'k8s-worker-3',
 'user-memcached': 'k8s-worker-1',
 'user-mention-service': 'k8s-worker-1',
 'user-mongodb': 'k8s-worker-1',
 'user-service': 'k8s-worker-2',
 'user-timeline-mongodb': 'k8s-worker-2',
 'user-timeline-redis': 'k8s-worker-1',
 'user-timeline

In [ ]:
'''if triggered migration decision
(1) measure traffic, and get the stress graph --> decide the pending the MS pods subgraph that needs to be migrated
(2) get the current cross-node dealy matrixs -->  return the quickest node chain (the max length is the depeth of the max call-graph?); 
(3) get the inactive pod list (for swapping); the inactive pods list, which is got by a classifer (SVM, or KNN), based on the cpu/memory/network_bd usages
  return the inactive pod list, along with the each pod's list;
  
(4) migrate pods (redeploy target microservices):
migrate the DM to the node of UM;

restrictions: sum_cpu_limits, sum_memory_limits 

if larger then the limits, find the swapping pod from inactive 

'''